# Setup

In [ ]:
%matplotlib inline

In [ ]:
from os.path import exists

import numpy as np
import pandas as pd
import requests
import seaborn as sns

# Checking existing findings
http://www.businessinsider.com/most-educated-places-map-2014-9

In [ ]:
base_url = 'http://api.census.gov/data/2012/acs5?get={}&for=place:*'

variables_old = {
    'NAME': 'place_name',
    'B01003_001E': 'population',
    'B15003_001E': 'population_over_25',
    'B15003_022E': 'bachelor',
    'B15003_023E': 'masters',
    'B15003_024E': 'professional',
    'B15003_025E': 'doctorate'
}

In [ ]:
def read_census(url):
    r = requests.get(url)
    data = r.json()
    return pd.DataFrame(data=data[1:], columns=data[0], dtype=int)

# Read data

In [ ]:
url = base_url.format(','.join(variables_old))
df_orig = (
    read_census(url)
    .rename(columns=variables_old)
    .sort_index(axis=1)
)

In [ ]:
df_orig.head()

# Determine bachelor+ attainment rate

In [ ]:
def bachelor_plus(df):
    return df[['bachelor', 'masters', 'professional', 'doctorate']].sum(axis=1)

In [ ]:
def pct_bachelor_plus(df):
    return df.bachelor_plus / df.population_over_25

In [ ]:
pbp_orig = (
    df_orig
    .assign(bachelor_plus=bachelor_plus)
    .assign(pct_bachelor_plus=pct_bachelor_plus)
)

In [ ]:
pbp_orig.plot.scatter('population_over_25', 'pct_bachelor_plus');

In [ ]:
highest_by_state_orig = (
    pbp_orig
    .loc[lambda df: df.population > 1000, :]
    .sort_values('pct_bachelor_plus', ascending=False)
    .groupby('state')
    .first()
    .sort_values('pct_bachelor_plus', ascending=False)
)

In [ ]:
highest_by_state_orig[['place_name', 'pct_bachelor_plus']].head()

In [ ]:
ax = pbp_orig.plot.scatter('population_over_25', 'pct_bachelor_plus', alpha=0.01, xlim=0)
highest_by_state_orig.plot.scatter('population_over_25', 'pct_bachelor_plus', color='red', ax=ax);

# Doing things the right way

In [ ]:
na_values = ('**', '-', '***', '*****', 'N', '(X)')

base_variables = {
    'HC01_{}_VC07': 'population_over_25',
    'HC01_{}_VC13': 'pct_bachelor',
    'HC01_{}_VC14': 'pct_graduate_or_professional'
}

estimate_variables = {var.format('EST'): name for var, name in base_variables.items()}
moe_variables = {var.format('MOE'): '{}_moe'.format(name) for var, name in base_variables.items()}
variables = {
    **{'GEO.id2': 'state_and_place', 'GEO.display-label': 'place_name'},
    **estimate_variables,
    **moe_variables
}

dtypes = {
    key: np.float16
    for key in variables
    if key.startswith('HC01_')
}

In [ ]:
def read_acs(year):
    filename = '../data/ACS_{year}_5YR_S1501/ACS_{year}_5YR_S1501_with_ann.csv'.format(year=str(year)[-2:])
    return (
        pd
        .read_csv(
            filename,
            usecols=variables.keys(),
            dtype=dtypes,
            na_values=na_values,
            encoding='latin1'
        )
        .rename(columns=variables)
)

In [ ]:
df_12 = read_acs(2012)

In [ ]:
df_12.population_over_25.value_counts()

In [ ]:
df_12.dtypes

In [ ]:
def state_and_place(df):
    state_and_place_str = df.state_and_place.astype(str).str.rjust(7)
    
    state = state_and_place_str.str[:2].astype(int).rename('state')
    place = state_and_place_str.str[2:].astype(int).rename('place')
    
    return df.join(state).join(place)

In [ ]:
def add_standard_error(df):
    """Add standard error columns"""
    ses = (
        df
        .filter(regex='_moe$', axis=1)
        .pipe(lambda df: df / 1.645)
        .rename(columns=lambda x: x[:-3] + 'se')
    )
    return df.join(ses)

In [ ]:
def pct_bachelor_plus(df):
    return df.pct_bachelor + df.pct_graduate_or_professional

In [ ]:
def pct_bachelor_plus_se(df):
    return (
        df
        [['pct_bachelor_se', 'pct_graduate_or_professional_se']]
        .pipe(np.square)
        .sum(axis=1)
        .pipe(np.sqrt)
    )

In [ ]:
def population_over_25_lower(df):
    lower = df.population_over_25 - df.population_over_25_moe
    lower[lower < 0] = 0
    return lower

In [ ]:
def pct_bachelor_plus_lower(df):
    moe = df.pct_bachelor_plus_se * 1.645
    lower = df.pct_bachelor_plus - moe
    # Set minimum of 0
    lower.loc[lower < 0] = 0
    
    return lower

In [ ]:
def make_pbp(df):
    return (
        df
        .pipe(state_and_place)
        .pipe(add_standard_error)
        .assign(
            pct_bachelor_plus=pct_bachelor_plus,
            pct_bachelor_plus_se=pct_bachelor_plus_se
        )
        .assign(
            population_over_25_lower=population_over_25_lower,
            pct_bachelor_plus_lower=pct_bachelor_plus_lower
        )
    )

In [ ]:
pbp_12 = df_12.pipe(make_pbp)

In [ ]:
pbp_12.head()

In [ ]:
pbp_12.plot.scatter('population_over_25_lower', 'pct_bachelor_plus_lower');

In [ ]:
def make_hbs(df):
    return (
        df
        .sort_values('pct_bachelor_plus_lower', ascending=False)
        .groupby('state')
        .first()
        .sort_values('pct_bachelor_plus_lower', ascending=False)
    )

In [ ]:
hbs_12 = make_hbs(pbp_12)

In [ ]:
hbs_12.head()

In [ ]:
ax = pbp_12.plot.scatter('population_over_25_lower', 'pct_bachelor_plus_lower', alpha=0.1)
hbs_12.plot.scatter('population_over_25_lower', 'pct_bachelor_plus_lower', ax=ax, color='red');

In [ ]:
pbp_12.describe()

In [ ]:
sns.jointplot('population_over_25_lower', 'pct_bachelor_plus_lower', data=pbp_12)